In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Define transforms to apply to each image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally with 50% probability
    transforms.RandomRotation(degrees=15),  # Randomly rotate images by up to 15 degrees
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

# Define your model, loss function, and optimizer as before
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Load your data as before
train_loader = DataLoader(datasets.ImageFolder('data/train', transform=transform), batch_size=32, shuffle=True)
val_loader = DataLoader(datasets.ImageFolder('data/test', transform=transform), batch_size=32, shuffle=False)

# Early stopping parameters
patience = 3  # Number of epochs to wait before stopping if no improvement
best_val_loss = float('inf')
patience_counter = 0

num_epochs = 50
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss}, Accuracy: {accuracy}%')

    # Check if validation loss improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # Reset the counter if we see an improvement
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1

    # If we've gone `patience` epochs without improvement, stop training
    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Epoch 1/50, Training Loss: 0.5528154820203781, Validation Loss: 0.2595035384098689, Accuracy: 95.1219512195122%
Epoch 2/50, Training Loss: 0.19555729404091834, Validation Loss: 0.10498437037070592, Accuracy: 98.78048780487805%
Epoch 3/50, Training Loss: 0.0807668499648571, Validation Loss: 0.06778828116754691, Accuracy: 97.5609756097561%
Epoch 4/50, Training Loss: 0.07116084545850754, Validation Loss: 0.05591379230221113, Accuracy: 97.5609756097561%
Epoch 5/50, Training Loss: 0.057587604597210885, Validation Loss: 0.06329512378821771, Accuracy: 97.5609756097561%
Epoch 6/50, Training Loss: 0.043369656428694725, Validation Loss: 0.05404382230093082, Accuracy: 96.34146341463415%
Epoch 7/50, Training Loss: 0.025599635485559703, Validation Loss: 0.045361838613947235, Accuracy: 98.78048780487805%
Epoch 8/50, Training Loss: 0.025660613924264906, Validation Loss: 0.042461615366240345, Accuracy: 98.78048780487805%
Epoch 9/50, Training Loss: 0.017249100748449563, Validation Loss: 0.0411631654327